# Question Answering

## PDF reader

https://archive.org/details/crossingthechasm_202002

In [ ]:
%pip install langchain openai chromadb tiktoken pypdf llama-index tqdm

In [1]:
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [2]:
import os
import tomli
import openai
with open('../.streamlit/secrets.toml','rb') as f:
    toml_dict = tomli.load(f)
openai.api_key = toml_dict['OPEN_AI_KEY']
os.environ['OPENAI_API_KEY'] = toml_dict['OPEN_AI_KEY']
# os.environ['PINECONE_API_KEY'] = toml_dict['PINECONE_API_KEY']
# os.environ['PINECONE_API_ENV'] = toml_dict['PINECONE_API_ENV']
os.environ['aws_access_key_id'] = toml_dict['aws']['aws_access_key_id']
os.environ['aws_secret_access_key'] = toml_dict['aws']['aws_secret_access_key']

In [4]:

# load document
loader = PyPDFLoader("../book/Crossing the Chasm.pdf")
documents = loader.load()

In [2]:
from llama_index import download_loader

PDFReader = download_loader("PDFReader")

loader = PDFReader()
documents = loader.load_data("../book/Crossing the Chasm.pdf")

### Connect to S3

In [5]:
import boto3
s3_client = boto3.client('s3',aws_access_key_id = os.environ['aws_access_key_id'],
                    aws_secret_access_key = os.environ['aws_secret_access_key'])
s3_bucket = 'book48'
object_name = 'index/index-full.json'
file_name = 'index-full.json'
s3_client.download_file(s3_bucket, object_name,file_name)

### Create index

In [23]:
from llama_index import LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI
llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

In [24]:
from llama_index import GPTSimpleVectorIndex

index = GPTSimpleVectorIndex.from_documents(documents,service_context=service_context)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 118920 tokens


In [29]:
index.service_context.llm_predictor.llm

ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x000001742FF3A560>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None)

In [25]:
query = "what is Vendor-Oriented Pricing?"
response = index.query(query)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 4226 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 9 tokens


In [31]:
print(response)

The original answer doesn't seem to be related to the context provided, therefore, it is better to output the original answer again. 

Original answer: 

Vendor-oriented pricing is a pricing strategy based on internal factors such as cost of goods, cost of sales, and promised rate of risk-adjusted return, among others. This strategy is critical for managing an enterprise profitably but has little immediate meaning in the marketplace unless it impacts other market-visible issues. Vendor-oriented pricing can also affect the number of transactions required to create a given amount of annual revenue. However, vendor-oriented pricing is considered the least sound basis for pricing decisions during the chasm period, as pricing decisions during this period should be almost entirely externally focused.


In [32]:
response.get_formatted_sources()

'> Source (Doc id: ae730396-8d19-4765-8042-34524fcfea0b): decisions are among the hardest for management groups\nto reach consensus on. The problem is that ...'

In [38]:
response.source_nodes[0].node.get_text()

'decisions are among the hardest for management groups\nto reach consensus on. The problem is that there are so many\nperspectives competing for the controlling influence. In this sec\xad\ntion we are going to sort out some of those perspectives and set\nout some rational guidelines for pricing during the chasm period.\nCustomer-Oriented Pricing\nThe first perspective to set on pricing is the customers’, and, as\nwe noted in the section on discovering the chasm, that varies\ndramatically with their psychographics. Visionaries—the cus\xad\ntomers dominating the early market’s development—are rela\xad\ntively price-insensitive. Seeking a strategic leap forward, with\nan order-of-magnitude return on investment, they are con\xad\nvinced that any immediate costs are insignificant when com\xad\npared with the end result. Indeed, they want to make sure there\nis, if anything, extra money in the price, because they know they\nare going to need special service, and they want their vendors to\nh

In [30]:
# save to disk
index.save_to_disk('../index/index.json')
# load from disk
index = GPTSimpleVectorIndex.load_from_disk('../index/index.json')

### Create Embeddings manually


In [ ]:
embed_model = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=embed_model
)

### Create Pinecone DB

In [ ]:
import pinecone 

pinecone.init(api_key=os.environ['PINECONE_API_KEY'], environment=os.environ['PINECONE_API_ENV']) 
# pinecone.list_indexes()
index = pinecone.Index('crossing-the-chasm') 

## QA

### load_qa_chain

In [16]:
# load document
loader = PyPDFLoader("book/Crossing the Chasm-202-217.pdf")
documents = loader.load()


In [14]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name='gpt-3.5-turbo')

chain = load_qa_chain(llm=chat, chain_type="map_reduce")
query = "what is Vendor-Oriented Pricing?"
chain.run(input_documents=documents, question=query)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=60).


'The given portion of the document does not provide a clear definition of Vendor-Oriented Pricing. However, it mentions that "vendor-oriented pricing represents the least sound basis for pricing decisions during the chasm period."'

### RetrievalQA

In [7]:
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [8]:
# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# select which embeddings we want to use
embeddings = OpenAIEmbeddings()
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)

Using embedded DuckDB without persistence: data will be transient


In [10]:
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":3})
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(
    llm=chat, chain_type="stuff", retriever=retriever, return_source_documents=True)
query = "what is Vendor-Oriented Pricing?"
result = qa({"query": query})

In [11]:
result

{'query': 'what is Vendor-Oriented Pricing?',
 'result': 'Vendor-oriented pricing is a pricing strategy that is based on internal factors such as the cost of goods, cost of sales, cost of overhead, cost of capital, promised rate of risk-adjusted return, and any number of other factors critical to managing an enterprise profitably on an ongoing basis. Its impact is on the number of transactions required to create a given amount of annual revenue. It sets the distribution channel decision by establishing a price-point ballpark that puts the product in the direct sales, web self-service, or sales 2.0 camp. The pricing strategy is not the most sound basis for pricing decisions during the chasm period, as it requires being almost entirely externally focused- both on the new demands of the mainstream customer and the new relationship you are trying to build with a mainstream channel.',
 'source_documents': [Document(page_content='208 Crossing the Chasm\nVendor -Oriented Pricing\nVendor-orien

In [12]:
result['result']

'Vendor-oriented pricing is a pricing strategy that is based on internal factors such as the cost of goods, cost of sales, cost of overhead, cost of capital, promised rate of risk-adjusted return, and any number of other factors critical to managing an enterprise profitably on an ongoing basis. Its impact is on the number of transactions required to create a given amount of annual revenue. It sets the distribution channel decision by establishing a price-point ballpark that puts the product in the direct sales, web self-service, or sales 2.0 camp. The pricing strategy is not the most sound basis for pricing decisions during the chasm period, as it requires being almost entirely externally focused- both on the new demands of the mainstream customer and the new relationship you are trying to build with a mainstream channel.'

### VectorstoreIndexCreator

Wrapper for the logic above

Source:

https://python.langchain.com/en/latest/modules/chains/getting_started.html
https://github.com/hwchase17/langchain/blob/master/langchain/indexes/vectorstore.py#L21-L74

In [ ]:
index = VectorstoreIndexCreator(
    # split the documents into chunks
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0),
    # select which embeddings we want to use
    embedding=OpenAIEmbeddings(),
    # use Chroma as the vectorestore to index and search embeddings
    vectorstore_cls=Chroma
).from_loaders([loader])
index.query(llm=chat, question=query, chain_type="stuff")

### ConversationalRetrievalChain

conversation memory + RetrievalQAChain

Allow for passing in chat history which can be used for follow up questions.

Source: https://python.langchain.com/en/latest/modules/chains/index_examples/chat_vector_db.html

In [ ]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# select which embeddings we want to use
embeddings = OpenAIEmbeddings()
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})
# create a chain to answer questions 
qa = ConversationalRetrievalChain.from_llm(chat, retriever)
chat_history = []
result = qa({"question": query, "chat_history": chat_history})

In [ ]:
result["answer"]

In [ ]:
chat_history = [(query, result["answer"])]
query = "How does it differ from Distribution-Oriented Pricing?"
result = qa({"question": query, "chat_history": chat_history})


In [ ]:
chat_history

In [ ]:
result['answer']